# Pretrained H&E classifier
Training set: central patches from Y-pos vs Y-neg images.  
No attempt to distinguish background, stroma, tumor.  
Retrain VGG.  
Expect no classification.

## Classify patches
Use VGG. 


In [1]:
import time
import os
#  os.environ['CUDA_VISIBLE_DEVICES'] = "0"   # turns off GPU?
import glob
#import cv2 # OpenCV-Python
#from PIL import Image
#Image.MAX_IMAGE_PIXELS = None
import numpy as np
import json
from tensorflow import keras
import keras.layers as kl
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

print('\n'.join(f'{m.__name__}=={m.__version__}' for m in globals().values() if getattr(m, '__version__', None)))

numpy==1.22.1
json==2.0.9
keras.api._v2.keras==2.7.0
pandas==1.4.0


In [2]:
DIR_IMAGES_IN = "/home/jrm/Martinez/images/HEcenterpatch/"  
PATCH_SIZE=224  # matches VGG
DIR_MODELS = "/home/jrm/Martinez/models/"
FILE_MODEL = "HE_Filter.026"
filepath=DIR_MODELS+FILE_MODEL
print("Will overwrite model at "+filepath)
BATCH_SIZE = 32
IMG_SIZE = (PATCH_SIZE,PATCH_SIZE)
CHANNELS = 3

Will overwrite model at /home/jrm/Martinez/models/HE_Filter.026


In [3]:
train_dir = os.path.join(DIR_IMAGES_IN, 'train')
valid_dir = os.path.join(DIR_IMAGES_IN, 'valid')
train_dataset = keras.utils.image_dataset_from_directory(
    train_dir, shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE)
valid_dataset = keras.utils.image_dataset_from_directory(
    valid_dir, shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE)


Found 30892 files belonging to 2 classes.
Found 7722 files belonging to 2 classes.


In [4]:
# Normalize color channel ints to floats (0,1)
# See https://www.tensorflow.org/tutorials/load_data/images

normalization_layer = kl.Rescaling(1.0/255.0)
train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
valid_dataset = valid_dataset.map(lambda x, y: (normalization_layer(x), y))

In [5]:
iterator = valid_dataset.take(10)
for x in iterator:
    print(x[0].shape)
    print(x[1])

(32, 224, 224, 3)
tf.Tensor([0 0 1 0 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 0 0], shape=(32,), dtype=int32)
(32, 224, 224, 3)
tf.Tensor([1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 0 0 1 1 1 0 1 1 0 0 1 1 1], shape=(32,), dtype=int32)
(32, 224, 224, 3)
tf.Tensor([0 0 1 0 0 0 0 1 1 0 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 0 0 1 1], shape=(32,), dtype=int32)
(32, 224, 224, 3)
tf.Tensor([1 1 1 1 1 1 1 1 1 0 0 1 0 1 0 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 0], shape=(32,), dtype=int32)
(32, 224, 224, 3)
tf.Tensor([1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1], shape=(32,), dtype=int32)
(32, 224, 224, 3)
tf.Tensor([0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1], shape=(32,), dtype=int32)
(32, 224, 224, 3)
tf.Tensor([1 1 1 0 0 0 0 0 0 0 1 0 1 0 1 1 1 0 1 1 1 1 1 0 1 1 0 1 1 0 1 1], shape=(32,), dtype=int32)
(32, 224, 224, 3)
tf.Tensor([1 0 1 1 0 0 1 1 1 1 1 1 1 1 0 1 1 1 0 0 1 1 0 1 0 1 0 1 1 0 0 1], shape=(32,), dtype=int32)
(32, 224, 224, 3)
tf.Tensor([0 0

## VGG
https://keras.io/guides/transfer_learning/  
https://towardsdatascience.com/step-by-step-guide-to-using-pretrained-models-in-keras-c9097b647b29  

In [6]:
CLASSES = 2
#base_model = VGG16(weights='imagenet',include_top=False)  # top= classification layer
# optional: input_shape=(224, 224, 3)
#base_model.trainable = False
#inputs = keras.Input(shape=(PATCH_SIZE, PATCH_SIZE, CHANNELS))
#x = base_model(inputs, training=False)

#x = keras.layers.GlobalAveragePooling2D()(x)
#outputs = keras.layers.Dense(CLASSES)(x)
#model = keras.Model(inputs, outputs)

#x = keras.layers.Flatten()(x)
#outputs = keras.layers.Dense(CLASSES)(x)
#model = keras.Model(inputs, outputs)

VGG = VGG16(weights='imagenet',include_top=False,input_shape=(PATCH_SIZE, PATCH_SIZE, CHANNELS))
VGG.trainable = False
model = keras.models.Sequential()
model.add(VGG)
model.add(kl.Flatten())
model.add(kl.Dense(16))
model.add(kl.Dropout(0.25))
model.add(kl.Dense(CLASSES,activation="softmax"))
model.summary()

ResourceExhaustedError: OOM when allocating tensor with shape[3,3,128,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:RandomUniform]

## Optimizer = SGD or ADAM ?
ADAM converges faster but SGD better in the long run.  
https://medium.com/geekculture/a-2021-guide-to-improving-cnns-optimizers-adam-vs-sgd-495848ac6008

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
           optimizer="sgd",
           metrics=["accuracy"])
EPOCHS=10
start = time.time()
hist = model.fit(train_dataset,validation_data=valid_dataset,epochs=EPOCHS)
end = time.time()
print("Elapsed time:",end-start)

In [ ]:
pd.DataFrame(hist.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0.0,1.0)
plt.show()

In [ ]:
model.save(filepath)

Note the portion of patches that are Ypos is  
35985 / (35985 + 10233) = 0.77859